In this assignement, feel free to use the `sparse` module from `scipy`.

Use the cell below for your imports.

In [1]:
from scipy import sparse
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

implement the function `mat_mul_coo` that takes two sparse matrices in `coo` and returns their product.

In [4]:
def mat_mul_coo(A, B):
    # Ensure the number of columns in A matches the number of rows in B
    if A.shape[1] != B.shape[0]:
        raise ValueError("Matrix dimensions don't match")

    # Convert A and B to CSR format for efficient computation
    A_csr = A.tocsr()
    B_csr = B.tocsr()

    # Initialize the result matrix in COO format
    rows = A_csr.shape[0]
    cols = B_csr.shape[1]
    data = []
    row_indices = []
    col_indices = []

    # Perform matrix multiplication
    for i in range(rows):
        row = A_csr.getrow(i)
        for j in range(cols):
            col = B_csr.getcol(j)
            dot_product = row.dot(col).sum()
            if dot_product != 0:
                data.append(dot_product)
                row_indices.append(i)
                col_indices.append(j)

    # Convert the result matrix to COO format and return it
    result = sparse.coo_matrix((data, (row_indices, col_indices)), shape=(rows, cols))
    return result


# Define two sparse matrices in COO format
A = coo_matrix(([1, 2, 3, 4, 5, 6], ([0, 0, 1, 1, 2, 2], [0, 1, 1, 2, 2, 3])), shape=(3, 4))
B = coo_matrix(([1, 2, 3, 4, 5, 6, 7, 8], ([0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 1, 2, 3, 2, 3, 4])), shape=(4, 5))

# Compute the product of A and B using our function
C = mat_mul_coo(A, B)

# Compute the expected result using NumPy
D = A.toarray().dot(B.toarray())

# Check that the result is correct
assert np.array_equal(C.toarray(), D)

implement the function `mat_mul_csr` that takes two sparse matrices in `csr` format and returns their product.

In [6]:
def mat_mul_csr(A, B):
    # Check if the number of columns in A matches the number of rows in B
    if A.shape[1] != B.shape[0]:
        raise ValueError("The number of columns in A must match the number of rows in B.")

    # Initialize arrays to store the output matrix in CSR format
    rows = []
    cols = []
    data = []
    ptrs = [0]

    # Loop over each row of A
    for i in range(A.shape[0]):
        # Loop over each column of B
        for j in range(B.shape[1]):
            dot_product = 0
            # Loop over the elements in the i-th row of A and the j-th column of B
            for k1 in range(A.indptr[i], A.indptr[i+1]):
                k2 = B.indptr[A.indices[k1]]
                while k2 < B.indptr[A.indices[k1]+1] and B.indices[k2] <= j:
                    if B.indices[k2] == j:
                        dot_product += A.data[k1] * B.data[k2]
                    k2 += 1
            # Add the result to the output matrix if it's nonzero
            if dot_product != 0:
                rows.append(i)
                cols.append(j)
                data.append(dot_product)
        # Update the pointer array for the current row
        ptrs.append(len(data))

    # Construct the output matrix in CSR format
    C = csr_matrix((data, (rows, cols)), shape=(A.shape[0], B.shape[1]))
    C.indptr = np.array(ptrs)

    return C


# Define two sparse matrices in CSR format
A = csr_matrix(([1, 2, 3, 4, 5, 6], ([0, 0, 1, 1, 2, 2], [0, 1, 1, 2, 2, 3])), shape=(3, 4))
B = csr_matrix(([1, 2, 3, 4, 5, 6, 7, 8], ([0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 1, 2, 3, 2, 3, 4])), shape=(4, 5))

# Compute the product of A and B using our function
C = mat_mul_csr(A, B)

# Compute the expected result using NumPy
D = A.toarray().dot(B.toarray())

# Check that the result is correct
assert np.array_equal(C.toarray(), D)

implement a function `solve_lin_sys` that takes a matrix `A` in `csr` format and a vector `b` as a numpy array and solves the system `Ax = b`.

In [7]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def solve_lin_sys(A, b):
    # Check if A is square
    if A.shape[0] != A.shape[1]:
        raise ValueError("A must be a square matrix.")
    
    # Check if the number of rows in A matches the length of b
    if A.shape[0] != len(b):
        raise ValueError("The number of rows in A must match the length of b.")
    
    # Solve the linear system using spsolve
    x = spsolve(A, b)
    
    return x

# Define a sparse matrix A in CSR format and a dense vector b
A = csr_matrix(([1, 2, 3, 4], ([0, 1, 2, 2], [0, 1, 1, 2])), shape=(3, 3))
b = np.array([1, 2, 3])

# Solve the linear system Ax = b using our function
x = solve_lin_sys(A, b)

# Print the solution vector x
print(x)


[1. 1. 0.]
